In [1]:

import pandas
from data_algebra.data_ops import *


# some example data
d1 = pandas.DataFrame({
    'ID': [2, 3, 7],
    'OP': ['A', 'B', 'B'],
})

d2 = pandas.DataFrame({
    'ID': [1, 1, 2, 3, 4, 2, 4, 4, 5, 5, 6],
    'OP': ['A', 'B', 'A', 'D', 'C', 'A', 'D', 'B', 'A', 'B', 'B'],
})


def merge_in_counts(pipeline, id_cols, new_table_descr):
    new_count_col = f'count_{new_table_descr.table_name}'
    return (
        pipeline.natural_join(
            b=new_table_descr
                .project({new_count_col: '(1).sum()'}, group_by=id_cols),
            by=id_cols,
            jointype='left',
            )
        .extend({new_count_col: f'{new_count_col}.coalesce_0()'})
    )


ops = data(d1=d1).use(merge_in_counts, ['ID'], data(d2=d2))

print(ops)

(
    TableDescription(table_name="d1", column_names=["ID", "OP"])
    .natural_join(
        b=TableDescription(table_name="d2", column_names=["ID", "OP"]).project(
            {"count_d2": "(1).sum()"}, group_by=["ID"]
        ),
        by=["ID"],
        jointype="LEFT",
    )
    .extend({"count_d2": "count_d2.coalesce(0)"})
)



In [2]:
ops.ex()

,ID,OP,count_d2
0,2,A,2.0
1,3,B,1.0
2,7,B,0.0
